# RAG Pipeline

deadline: 16 декабря 2025, 23:59

В последние годы подход Retrieval-Augmented Generation (RAG) стал одним из ключевых методов повышения качества ответов больших языковых моделей. В этой работе вы должны пошагово реализовать собственную RAG-систему “с нуля”, используя открытые модели и инструменты: HuggingFace, FAISS и классические трансформеры для генерации текста.

## ПРАВИЛА
Домашнее задание выполняется в группе до 4-х человек.
Домашнее задание оформляется в виде отчета в jupyter-тетрадке.
Отчет должен содержать: имена всех членов группы, нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали.
Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
Плагиат и любое недобросовестное цитирование приводит к обнулению оценки.
За каждую неделю просрочки после дедлайна начисляет по 1 штрафному баллу (например, при дедлайне 01.10.25 в 23:59 сдача ДЗ 12.10.25 ведёт к 2 штрафным баллам).
Бонусные баллы позволяют повысить общую оценку за ДЗ до максимальной (если были ошибки или недочеты, повлекшие снижение баллов).

In [1]:
!pip install datasets --quiet
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
raw_text = "\n".join(dataset["train"]["text"])

corpus = raw_text

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Часть 1. Подготовка данных: разбиение на чанки [2 балла]

Необходимо реализовать функцию `make_chunks(text: str, chunk_size: int, overlap: int) -> List[str]`, которая делит длинный текст на перекрывающиеся фрагменты. Рекомендуемый размер одного чанка — 250–350 символов, а перекрытие — 40–60 символов. Попробуйте несколько комбинаций параметров. После реализации примените функцию к `corpus`, получите список `all_chunks` и выведите общее количество чанков, а также первые два и последние два фрагмента.

Затем вычислите для `all_chunks` среднюю, минимальную и максимальную длину чанков. На основе этих значений сделайте вывод, стоит ли изменять `chunk_size` или `overlap`, и достаточно ли полученные фрагменты подходят для смыслового поиска в RAG-системе.

Также ответьте в тексте:
*зачем нужно перекрытие чанков в RAG и что произойдёт, если сделать `overlap = 0`?*


## Часть 3. Создание эмбеддингов [2.5 балла]

Теперь нужно закодировать их в векторное представление. Мы будем использовать модель эмбеддингов, например
**`sentence-transformers/all-MiniLM-L6-v2`** — она достаточно быстрая и отлично подходит для экспериментов в Colab.

Сначала загрузите модель при помощи `sentence-transformers` (или вручную через `transformers` + `AutoModel`). Затем напишите небольшую функцию `embed_text(text: str) -> np.ndarray`. Примените функцию ко всем чанкам из Части 2. В результате у вас должен получиться массив эмбеддингов. Выведите пример эмбеддинга.

Чтобы улучшить работу поиска (особенно при cosine similarity), нормализуйте все полученные эмбеддинги до единичной длины и сохраните результат в `embeddings_normalized`.

Попробуйте немного проанализировать сами эмбеддинги. Посчитайте среднее расстояние (L2 или cosine) между случайными парами чанков и соседними чанками (i и i+1). Сравните результаты. Насколько соседние чанки действительно ближе друг к другу? Помогает ли перекрытие при разбиении текста сформировать более «плавное» и последовательное векторное пространство?

## Часть 4. Построение поискового индекса и реализация поиска [2 балла]

Теперь, когда все эмбеддинги получены, пора собрать из них настоящий поисковый индекс. Мы будем использовать FAISS — быстрый и удобный инструмент для поиска ближайших векторов.

Сначала подключите FAISS, определите размерность эмбеддингов и создайте индекс. Можно выбрать один из двух вариантов:
* **IndexFlatL2**, если хотите искать ближайших соседей по евклидову расстоянию;
* **IndexFlatIP**, если используете нормализованные векторы и хотите фактически искать по cosine similarity.

Добавьте в него все эмбеддинги (`embeddings_normalized`). Ответьте: сколько векторов добавлено, какая размерность у индекса, чем отличается поиск по L2 от cosine similarity, и почему вы выбрали свой вариант.

Далее нужно реализовать простую функцию поиска:

```python
def search(query: str, k: int = 5):
    ...
```

Функция должна закодировать текст запроса с помощью `embed_text()`, нормализовать вектор, отправить его в FAISS (`index.search`) и вернуть найденные чанки по их индексам.

Проверьте, как работает поиск, выполнив запрос по вашему выбору.

А затем сформулируйте три собственных вопроса по содержанию корпуса, выполните по каждому поиск top-3 чанков и для каждого результата дайте небольшую оценку релевантности:
0 — не похоже,
1 — отдалённо связано,
2 — действительно по теме.

In [4]:
!pip install faiss-gpu-cu12

## Часть 5. Генерация ответов с помощью модели [2.5 балла]

Теперь пора собрать полноценный RAG и научиться генерировать ответы.
Для этого мы подключим небольшую, бесплатную модель вроде `google/flan-t5-small` (или любую другую модель на ваш выбор).

Сначала нужно придумать функцию:

```python
build_prompt(query: str, context_chunks: List[str]) -> str
```

Эта функция должна собирать промпт для модели: формулировка вашего вопроса + добавленные кусочки текста, которые модель должна использовать. Стиль промпта — на ваше усмотрение. Можно попробовать несколько вариантов: более строгий, более разговорный, с ограничениями («используй только этот контекст»), и посмотреть, что получится.

После этого напишите функцию генерации:

```python
generate_answer(prompt: str, max_new_tokens: int) -> str
```

Проверьте её на простом тестовом примере, чтобы убедиться, что всё работает.

Теперь можно собрать простой RAG-пайплайн:

```python
def rag(query: str, k: int = 3) -> str:
    found_chunks = search(query, k=k)
    prompt = build_prompt(query, found_chunks)
    answer = generate_answer(prompt)
    return answer
```

Поиграйте с параметром `k` (например, 1, 3, 5, 7) и посмотрите, как меняется качество ответа.

Далее задайте **три своих вопроса** по содержанию корпуса, запустите `rag()` на каждом из них и оцените качество ответа:

* **0** — ответ нерелевантен, модель галлюцинирует или игнорирует контекст
* **1** — что-то уловила, но не полностью
* **2** — чёткий, точный ответ, построенный на найденных чанках


## Часть 6. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Что вы сделали на каждом этапе (разбиение текста на чанки, создание эмбеддингов, построение индекса, генерация ответов)? Какие решения сработали хорошо, а что можно улучшить? Как в целом работает собранная вами RAG-система и насколько качественно она отвечает на вопросы?